In [1]:
!pip install torch_geometric

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.10.2
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://

In [3]:
import os
import torch
import matplotlib.pyplot as plt

from torch_geometric.nn import GCNConv
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import NormalizeFeatures
import torch_geometric.transforms as T

transform = T.Compose([T.GCNNorm(), T.NormalizeFeatures()])

dataset = TUDataset(root="data/MUTAG", name="MUTAG", transform=None)
print(dataset)
data = dataset[0]
print(data)

MUTAG(188)
Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])


In [4]:
print(f'Number of classes: {dataset.num_classes}')

Number of classes: 2


In [5]:
import numpy as np
torch.manual_seed(12345)
dataset = dataset.shuffle()

split = int(0.8 * len(dataset))
train_dataset, test_dataset = dataset[:split], dataset[split:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 150
Number of test graphs: 38


In [6]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2636], x=[1188, 7], edge_attr=[2636, 4], y=[64], batch=[1188], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2506], x=[1139, 7], edge_attr=[2506, 4], y=[64], batch=[1139], ptr=[65])

Step 3:
Number of graphs in the current batch: 22
DataBatch(edge_index=[2, 852], x=[387, 7], edge_attr=[852, 4], y=[22], batch=[387], ptr=[23])



In [7]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [8]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 31):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 002, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 003, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 004, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 005, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 006, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 007, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 008, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 009, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 010, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 011, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 012, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 013, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 014, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 015, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 016, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 017, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 018, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 019, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 020, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 021, Train Acc: 0.6467, Test Acc:

In [9]:
for data in test_loader:
  out = model(data.x, data.edge_index, data.batch) 
  print(F.softmax(out[0], dim=0)[0] > 2) 
  pred = out.argmax(dim=1) 
  print(pred[0].item()) 
  print(len(pred))
  break

tensor(False)
1
38


In [25]:
from re import sub
from sklearn import neighbors
import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GCNConv, GNNExplainer

from torch_geometric.loader import DataLoader

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import subgraph
import numpy as np

import networkx as nx
from torch_geometric import utils

def expand(starts, ends, target, max_depth=1, prev=[]):
    node_neighbors = np.array([ends[idx] for idx, node in enumerate(starts) if node == target and ends[idx] != target])
    prev.append(target)
    if max_depth > 1:
        for n in node_neighbors:
            node_neighbors = np.concatenate((node_neighbors, expand(starts, ends, target=n, max_depth=max_depth-1, prev=prev)), axis=0)
    indices = np.unique(node_neighbors, return_index=True)[1]
    return np.array([node_neighbors[i] for i in sorted(indices)])


def process_one_graph(data):
    used = []
    num_nodes = data.x.shape[0]
    subgraph_sizes = [int(num_nodes/n) for n in range(2, 5)]
    start_nodes, end_nodes = np.array(data.edge_index)
    sub_graphs = []
    # make a grow from each node
    for target_idx in range(num_nodes):
        nodes_to_keep = expand(starts=start_nodes, ends=end_nodes, target=target_idx, max_depth=3, prev=[])
        if nodes_to_keep.shape[0] == 0:
            continue
        for size in subgraph_sizes:
            # select the grows based on size
            _subset = nodes_to_keep[:size]
            # remove repetitive stuff
            if set(_subset) in used:
                continue
            else:
                used.append(set(_subset))
            _subset = torch.from_numpy(np.array(_subset))
            sub_graphs.append((target_idx, data.subgraph(subset=_subset)))
    return sub_graphs



dataset = 'MUTAG'
path = osp.join(osp.dirname(osp.realpath("__file__")), '..', 'data', 'TUDataset')
transform = T.Compose([T.GCNNorm(), T.NormalizeFeatures()])
dataset = TUDataset(path, dataset, transform=None)

pred_list = []
all_subgraphs = []
confidences = []
for idx, data in enumerate(tqdm.tqdm(dataset)):
    out = model(data.x, data.edge_index, data.batch)  
    out = out.softmax(dim=1)
    pred = out.argmax(dim=1)
    pred_list.append(pred)

    subgraphs = process_one_graph(data)
    all_subgraphs.append(subgraphs)
    confidences.append(out[0, pred].item())


# for target_idx, graph in tqdm.tqdm(all_subgraphs):
#     # nx.draw_networkx(utils.to_networkx(graph, remove_self_loops=True))
#     # plt.show()


100%|██████████| 188/188 [00:02<00:00, 78.63it/s]

[0.5812622308731079, 0.579557478427887, 0.5795118808746338, 0.5815139412879944, 0.5746147036552429, 0.5783131122589111, 0.581076443195343, 0.5801668167114258, 0.5779090523719788, 0.5811891555786133, 0.5812942385673523, 0.5808764100074768, 0.5801395773887634, 0.5780375599861145, 0.5814969539642334, 0.5801562666893005, 0.5776200294494629, 0.580407440662384, 0.5794845223426819, 0.5805829763412476, 0.5793904662132263, 0.5802112817764282, 0.5818279385566711, 0.582056999206543, 0.5803793668746948, 0.5763154625892639, 0.5818226933479309, 0.5794278979301453, 0.5780917406082153, 0.581822395324707, 0.5802112221717834, 0.577476441860199, 0.5766609311103821, 0.5806207060813904, 0.5804061889648438, 0.5807485580444336, 0.5812678933143616, 0.5809884667396545, 0.5790954828262329, 0.5806484818458557, 0.5794982314109802, 0.5780047178268433, 0.5814948678016663, 0.5812622308731079, 0.5774005651473999, 0.5818226933479309, 0.5801165103912354, 0.5812565088272095, 0.5783689022064209, 0.5789394378662109, 0.580

In [11]:
print(type(all_subgraphs))
for data in all_subgraphs:
  print(data[1])
  break



<class 'list'>
(0, Data(edge_index=[2, 8], x=[5, 7], edge_attr=[8, 4], y=[1]))


In [12]:
print(type(test_dataset))
for test_data in test_dataset:
  print(test_data)
  break

<class 'torch_geometric.datasets.tu_dataset.TUDataset'>
Data(edge_index=[2, 50], x=[22, 7], edge_attr=[50, 4], y=[1])


In [40]:
min_sufficient_explanation = [] #list of lists, each inner list correspond to one full graph in TUDataset Protein,
#containing each full graph's MSE's
threshold = 0.58
for idx, data_collection in enumerate(tqdm.tqdm(all_subgraphs)):
  sub_mse = np.array([]) #sub_mse for this collection, correspond to one original full graph
  sub_confidence = []
  for data in data_collection:
    out = model(data[1].x, data[1].edge_index, data[1].batch)

    out = out.softmax(dim=1)
    pred = out.argmax(dim=1)
    
      
    # if the subgraph's predicted label matches the original full graph's predicted, add to its min_suff_expl list
    if torch.eq(pred, pred_list[idx]).item():
      sub_mse = np.append(sub_mse, data)
      #sub_mse.append(data)
      sub_confidence.append(abs((out[0, pred]).item() - confidences[idx]))
  sort_indices = np.argsort(sub_confidence)
  sub_mse = sub_mse[sort_indices]
  
  min_sufficient_explanation.append(sub_mse)

  

100%|██████████| 188/188 [00:07<00:00, 26.20it/s]


In [41]:
for i in range(10):
  print(len(min_sufficient_explanation[i]))
  print(len(all_subgraphs[i]))
  print(len(min_sufficient_explanation[i]) < len(all_subgraphs[i]))
for data in all_subgraphs:
  print(data)
  break

for data in min_sufficient_explanation:
  print(len(data))
  break

#for example, the first three lines of outputs
#means 35 of the 43 subgraphs of the 0-th full graph
#are min sufficient explanations

43
43
False
32
32
False
32
32
False
44
44
False
28
28
False
56
56
False
37
37
False
46
46
False
29
29
False
43
43
False
[(0, Data(edge_index=[2, 14], x=[7, 7], edge_attr=[14, 4], y=[1])), (0, Data(edge_index=[2, 8], x=[5, 7], edge_attr=[8, 4], y=[1])), (0, Data(edge_index=[2, 6], x=[4, 7], edge_attr=[6, 4], y=[1])), (1, Data(edge_index=[2, 14], x=[7, 7], edge_attr=[14, 4], y=[1])), (1, Data(edge_index=[2, 8], x=[5, 7], edge_attr=[8, 4], y=[1])), (2, Data(edge_index=[2, 16], x=[8, 7], edge_attr=[16, 4], y=[1])), (2, Data(edge_index=[2, 6], x=[4, 7], edge_attr=[6, 4], y=[1])), (3, Data(edge_index=[2, 8], x=[5, 7], edge_attr=[8, 4], y=[1])), (3, Data(edge_index=[2, 2], x=[4, 7], edge_attr=[2, 4], y=[1])), (4, Data(edge_index=[2, 14], x=[8, 7], edge_attr=[14, 4], y=[1])), (4, Data(edge_index=[2, 8], x=[5, 7], edge_attr=[8, 4], y=[1])), (4, Data(edge_index=[2, 2], x=[4, 7], edge_attr=[2, 4], y=[1])), (5, Data(edge_index=[2, 6], x=[4, 7], edge_attr=[6, 4], y=[1])), (6, Data(edge_index=[2, 14

In [15]:
#input: 
# pred_list: [tensor, tensor, tensor ...], len = len(TUDataset/Protein)
#   each element is a one-element tensor 
#   representing the predicted label of one graph in TUDataset/Protein
# all_subgraph: [[subgraph, subgraph..], [subgraph, subgraph..]...], len = len(TUDataset/Protein)
#   each element is a list of subgraphs obtained from one graph in TUDataset/Protein
#   each subgraph is a tuple in the form (k, Data(edge_index=[2, 24], x=[8, 0], y=[1]))
#   where k is the index of the node that this subgraph expanded from
#output:
# min_sufficient_explanation: [[subgraph, subgraph,..], [subgraph, subgraph]...]
#   len = len(TUDataset/Protein)
#   each element is a list of subgraphs which are the MSEs
#   each subgraph is a tuple in the form (k, Data(edge_index=[2, 24], x=[8, 0], y=[1]))
#   where k is the index of the node that this subgraph expanded from